# MODEL TRAINING - testing the full approach
saving models in a different folder using pickle, saving them in the dataframe causes xgboost to crash

**DO NOT RUN AGAIN**

In [1]:
#external libraries
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.colors as clt
import plotly
import plotly.subplots as sb
import plotly.express as px
import plotly.graph_objects as go
import dotenv
import pandas as pd
import scipy.fft as fft
import scipy.signal as sg
import scipy.io as sio
import pickle as pkl
import xgboost as xgb
import time
import uuid

#project library
from spinco import *

#environment variables
dotenv.load_dotenv('lab.env')

#project variables
datapath=os.environ['DATAPATH']
cognipath=datapath+"/COGNITION"
dreamspath=datapath+"/DREAMS"
masspath=datapath+"/MASS"


## experiment id
we'll use it to create a folder at the end of the script

In [2]:
experimentId=str(uuid.uuid4())

## define a fixed samplerate

In [3]:
samplerate=200

## load mass

In [4]:
#annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate,onlySpindlesFilteredN2=True)
annotations, signalsMetadata = loadMASSSpindles(masspath,forceSamplerate=samplerate)

In [5]:
annotations=annotations[annotations.type=='spindle'].reset_index(drop=True)
len(annotations)

33458

In [6]:
minDuration=0.3
maxDuration=5
annotations=annotations[annotations.duration>minDuration].reset_index(drop=True)
annotations=annotations[annotations.duration<maxDuration].reset_index(drop=True)
len(annotations)

33269

In [7]:
annotations.head()

,type,expert,subjectId,labelerId,startTime,duration,samplerate,stopTime,startInd,stopInd
0,spindle,E1,0001,0001,888.327805,0.640579,200,888.968384,177666,177794
1,spindle,E1,0001,0001,905.758061,0.578094,200,906.336155,181152,181267
2,spindle,E1,0001,0001,917.731574,0.847603,200,918.579177,183546,183716
3,spindle,E1,0001,0001,922.078189,0.878845,200,922.957034,184416,184591
4,spindle,E1,0001,0001,939.055445,0.757767,200,939.813212,187811,187963


In [8]:
signalsMetadata.head()

,subjectId,file,channel,duration,samplerate,isOriginalSamplerate,database
0,0001,MASS_0001.pkl,C3-CLE,28956.0,200,False,MASS
1,0002,MASS_0002.pkl,C3-CLE,35016.0,200,False,MASS
2,0003,MASS_0003.pkl,C3-CLE,36760.0,200,False,MASS
3,0004,MASS_0004.pkl,C3-CLE,28004.0,200,False,MASS
4,0005,MASS_0005.pkl,C3-CLE,31244.0,200,False,MASS


## define a fixed feature selection to use

In [9]:
featureSelection=loadPickle("featureSelection_900bootstraping_iterations.pkl")
featureSelection

,window,characteristic,bandName,Times_selected
0,2,sigmaIndex,broadband,1000.0
1,2,hjortActivity,sigma,1000.0
2,1.5,sigmaIndex,broadband,1000.0
3,1,hjortActivity,sigma,1000.0
4,1,sigmaIndex,broadband,1000.0
5,2,hjortActivity,beta1,1000.0
6,2,hjortActivity,beta2,1000.0
7,1.5,hjortActivity,sigma,999.0
8,2,hjortMobility,beta1,994.0
9,2,hjortActivity,delta2,986.0


## load data split
10 fold CV with 4 subjects for testing

In [10]:
#by now we used fixed CV data splits instead
dataSplits=loadPickle("dataSplits_LOOCV.pkl")
dataSplits

,train,val,test
0,"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001
1,"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002
2,"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003
3,"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005
4,"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006
5,"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007
6,"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009
7,"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010
8,"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011
9,"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012


## define annotation criteria
we want to test the models in three diferent ways

In [11]:
annotationCriteria=pd.DataFrame({
    'criteriumId':['0001','0002','0003'],
    'criteriumName':['E1','E2','union'],
    'labelerIdList':[['0001'],['0002'],['0001','0002']]
})

In [12]:
annotationCriteria

,criteriumId,criteriumName,labelerIdList
0,0001,E1,[0001]
1,0002,E2,[0002]
2,0003,union,"[0001, 0002]"


In [13]:
experimentModels=pd.merge(annotationCriteria,dataSplits,how='cross')

In [14]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012


## model fitting

In [15]:
n_jobs=4 # paralelization parameter for xgboost (keep it 8 as maximum)
learning_rate=0.4
subsample=0.6

num_boost_round=60  #number of boosting rounds per model

models=[]
modelIds=[]
valF1s=[]
spindleTimeRates=[]
for index, row in experimentModels.iterrows():
    print('**********************')
    print(str(index+1)+' of '+str(len(experimentModels)))
    #Define annotations criterium
    usedAnnotations=annotations[annotations.labelerId.isin(row.labelerIdList)].reset_index(drop=True)
    #Features
    trainFeatures=loadFeatureMatrix(row.train,featureSelection,signalsMetadata,samplerate,datapath)
    #Labels
    trainLabels=loadLabelsVector(row.train,usedAnnotations,signalsMetadata,samplerate)

    """ # Restrict trainingto N2 --------------------------------------------------------------->
    maskVector=loadN2maskVector(row.train,hypnograms,signalsMetadata,samplerate)
    trainFeatures=trainFeatures[maskVector,:]
    trainLabels=trainLabels[maskVector]
    #<--------------------------------------------------------------------------------------- """   #not in this experiment
    
    #Train the models
    params={
        'n_jobs':n_jobs,
        'learning_rate':learning_rate,
        'subsample':subsample,
        'objective':'binary:logistic'
    }
    trainDMatrix=xgb.DMatrix(data=trainFeatures,label=trainLabels)
    xgb_model = xgb.train(params,trainDMatrix,num_boost_round=num_boost_round)
    spindleTimeRates.append(sum(trainLabels)/len(trainLabels))
    models.append(xgb_model)
    modelIds.append(str(uuid.uuid4()))
    del trainFeatures, trainLabels, usedAnnotations, xgb_model, trainDMatrix
        

**********************
1 of 45
**********************
2 of 45
**********************
3 of 45
**********************
4 of 45
**********************
5 of 45
**********************
6 of 45
**********************
7 of 45
**********************
8 of 45
**********************
9 of 45
**********************
10 of 45
**********************
11 of 45
**********************
12 of 45
**********************
13 of 45
**********************
14 of 45
**********************
15 of 45
**********************
16 of 45
**********************
17 of 45
**********************
18 of 45
**********************
19 of 45
**********************
20 of 45
**********************
21 of 45
**********************
22 of 45
**********************
23 of 45
**********************
24 of 45
**********************
25 of 45
**********************
26 of 45
**********************
27 of 45
**********************
28 of 45
**********************
29 of 45
**********************
30 of 45
**********************
31 of 45
*****************

In [16]:
experimentModels["modelId"]=modelIds
experimentModels["spindleTimeRate"]=spindleTimeRates

In [17]:
experimentModels

,criteriumId,criteriumName,labelerIdList,train,val,test,modelId,spindleTimeRate
0,0001,E1,[0001],"[0003, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0017, 0002, 0006]",0001,32f08b99-e2ed-4b74-b08c-2735d9bb23a4,0.018639
1,0001,E1,[0001],"[0001, 0003, 0005, 0007, 0009, 0010, 0011, 001...","[0006, 0013, 0019]",0002,6334de73-7d38-4b21-b2e9-f3b61bf6acd6,0.020055
2,0001,E1,[0001],"[0002, 0005, 0007, 0009, 0010, 0011, 0012, 001...","[0013, 0006, 0001]",0003,aebdf3f1-ad7f-45ed-8fc5-d11e7f394f9b,0.021075
3,0001,E1,[0001],"[0001, 0002, 0003, 0006, 0007, 0009, 0010, 001...","[0018, 0014, 0012]",0005,738ba2b7-5545-4ab9-867a-1f086d1d33b7,0.018331
4,0001,E1,[0001],"[0003, 0007, 0009, 0010, 0011, 0012, 0013, 001...","[0005, 0001, 0002]",0006,f12eb2ac-107c-4965-8b30-7e3cfcbee2b4,0.019222
5,0001,E1,[0001],"[0001, 0002, 0003, 0010, 0011, 0012, 0013, 001...","[0005, 0006, 0009]",0007,08efaffe-20f5-40ff-a941-617cf20394ac,0.019625
6,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0007, 0010, 0011, 001...","[0002, 0017, 0013]",0009,d0536ae7-5f23-4dcd-ab20-d62f79dda03c,0.017824
7,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0007, 0009, 0012, 001...","[0017, 0001, 0011]",0010,5136d7b9-621b-44ae-a93c-ce25b85baf89,0.017542
8,0001,E1,[0001],"[0001, 0003, 0005, 0006, 0009, 0010, 0012, 001...","[0002, 0018, 0007]",0011,bba7103e-b894-4ee6-b96c-5fe5c12b08c5,0.015442
9,0001,E1,[0001],"[0002, 0003, 0005, 0006, 0009, 0011, 0013, 001...","[0001, 0007, 0010]",0012,aa71ecc6-ba7d-4a68-af7c-37927934aefa,0.016460


## save results

In [18]:
#create parent folder
experimentpath=datapath+"/experiments/"+experimentId
os.mkdir(experimentpath)
#save each of the models
for id, model in zip(modelIds,models):
    model.save_model(experimentpath+"/"+id+".json")
#save experiment information
dumpPickle(experimentpath+"/experimentModels.pkl",experimentModels)
dumpPickle(experimentpath+"/featureSelection.pkl",featureSelection)

In [19]:
#you need to use this in the following parts of the experiment
print(experimentId)

e00fe93e-07e4-4b7e-9c01-68c740330512
